### Script to produce the ReLi Corpus with PALAVRAS parser dependency output

Using PALAVRAS revision 10754, compiled on 2015-06-06.


Corpus reference:
 
          Freitas, C., Motta, E., Milidiú, R., & Cesar, J. (2012).
          Vampiro que brilha... rá! Desafios na anotação de opinião em um corpus
          de resenhas de livros. Proceedings do XI Encontro de Linguística de Corpus (XI ELC). São Carlos - SP.
          http://www.linguateca.pt/Repositorio/ReLi/

Parser reference:

          Bick, Eckhard (2000), The Parsing System "Palavras" - Automatic Grammatical 
          Analysis of Portuguese in a Constraint Grammar Framework
          Aarhus: Aarhus University Press -- dr.phil. thesis
          http://beta.visl.sdu.dk/constraint_grammar.html

### Final XML format style
We are going to PALAVRAS new tags to 'tokens'. The parser can provide morf, extra, head, deprel and srl.

```xml
<Reviews>
    <Review rid="">
        <sentences>
            <sentence id="">
                <text>....</text>
                <Opinions>
                    <Opinion target="" category="" polarity="" from="" to=""/>
                </Opinions>
                <tokens>
                    <word id="" form="" from="" to="" base="" morf="" postag="" obj="" opinion="" morf="" extra="" head="" deprel="" srl="" />
                </tokens>
            </sentence>
        </sentences>
    </Review>
</Reviews>
```


## 1. Testing PALAVRAS communication

Palavras is a proprietary syntatic parser based on a constraint grammar. The parser used on these experiments was installed on the path: /opt/palavras

In [1]:
from subprocess import Popen, PIPE
import os

# necessary to PALAVRAS
os.environ['PERL_UNICODE'] = 'SDA'

# option --role outputs also the semantic role label
PALAVRAS_CMD = ['/opt/palavras/por.pl', '--role']

sentence_string = 'O menino jogou bola.'

# Send the string to be parsed with PALAVRAS
p = Popen(PALAVRAS_CMD, stdin=PIPE, stdout=PIPE, stderr=PIPE)
(stdout, stderr) = p.communicate(input=sentence_string.encode('utf8'))

output = stdout.decode('utf8')
print (output)


O 	 [o] <*> <artd> DET M S @>N #1->2 
menino 	 [menino] <Hbio> N M S @SUBJ> §AG #2->3 
jogou 	 [jogar] <fmc> <vH> <mv> V PS 3S IND VFIN @FS-STA §PRED #3->0 
bola 	 [bola] <cc> <tool> <food-c-h> <act> N F S @<ACC §TH #4->3 
$.  PU @PU #5->0 
</s>




The format above is the VISL format and it is rather difficult to process. 
In order to get the output in a better format we are going to use some 
pos-processing perl scripts provided in PALAVRAS to output in MALT parser XML format.

From palavras documentation:

    ##############################################################################                                                                        
    ############## PALAVRAS 3.0, version 2015 ###########################
    ############## (c) Eckhard Bick & GrammarSoft Aps ############################
    ##############################################################################

    last updated: January 2015

    ################################################################################

    How to run the parser ....

    in the current install folder, with utf-8 input:

    ### dependency trees
        (this is the standard version, and also the default without parameters)

        cat txt | /opt/palavras/por.pl --dep (OR just: cat txt | /opt/palavras/por.pl)
        or in combination with --sem: | /opt/palavras/bin/cg2dep pt 

        *** in MALT-xml format: | /opt/palavras/bin/visldep2malt.pl | /opt/palavras/bin/extra2sem

    ### state of the art version, full analysis, with noun- and name-semantics

        cat txt | /opt/palavras/por.pl --sem

    ### with semantic roles: cat txt | /opt/palavras/por.pl --role


In [2]:
PALAVRAS_CMD = ['/opt/palavras/por.pl', '--role']
PALAVRAS_MALT = ['/opt/palavras/bin/visldep2malt.pl']
PALAVRAS_EXTRA2SEM = ['/opt/palavras/bin/extra2sem']

# PALAVRAS parser
p = Popen(PALAVRAS_CMD, stdin=PIPE, stdout=PIPE, stderr=PIPE)
(stdout, stderr) = p.communicate(input=sentence_string.encode('utf8'))

# the script to convert from visl format to malt format
p = Popen(PALAVRAS_MALT, stdin=PIPE, stdout=PIPE, stderr=PIPE)
(stdout, stderr) = p.communicate(input=stdout)

# script to keep only semantic information in the xml
p = Popen(PALAVRAS_EXTRA2SEM, stdin=PIPE, stdout=PIPE, stderr=PIPE)
(stdout, stderr) = p.communicate(input=stdout)

output = stdout.decode('utf8')
print(output)

<?xml version="1.0" encoding="UTF-8"?>
<treebank id="Arboretum">
<body>

	<word id="1" form="O" base="o" postag="pron-indef" morf="M S" extra="* " head="2" deprel="DN"/>
	<word id="2" form="menino" base="menino" postag="n" morf="M S" sem="Hbio" extra="--" head="3" deprel="S §AG"/>
	<word id="3" form="jogou" base="jogar" postag="v-fin" morf="PS 3S IND VFIN" extra="fmc vH mv" head="0" deprel="STA §PRED"/>
	<word id="4" form="bola" base="bola" postag="n" morf="F S" sem="cc tool food-c-h act" extra="--" head="3" deprel="Od §TH"/>
	<word id="5" form=". PU @PU" base="--" postag="pu" morf="--" extra="--" head="0" deprel="PU"/>

</sentence>
</body>
<head>
        <annotation>
                <attribute name="form"/>
		<attribute name="lemma"/>
		<attribute name="head"/>
                <attribute name="postag">
                      <value name="n"/>
                      <value name="pron-indef"/>
                      <value name="pu"/>
                      <value name="v-fin"/>
           

However, these scripts has a lot of bugs. I tried to fix them with the function below.

In [3]:
# Lets use logging to show any problem
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

In [4]:
from lxml import etree
from lxml.etree import XMLSyntaxError
import re

# receive a xml string and return a etree xml element with possible bugs fixed
def fix_palavras_malt(palavras_xml):
    
    #remove <head></head> apart
    palavras_xml = re.sub(r'<head>.*</head>', '', palavras_xml, flags=re.DOTALL)
    
    # fix missing open <sentence> in xml
    if palavras_xml.find('<body>\n</body>') == -1:
        palavras_xml = palavras_xml.replace('<body>', '<body>\n<sentence>')

    # fix missing open <sentence> in xml between break sentences
    palavras_xml = re.sub(r'/>[\n\t ]+<word id="1"',
                    '/>\n</sentence>\n<sentence>\n<word id="1"', palavras_xml, re.M)

    # tags lixo
    palavras_xml = re.sub(r'<lixo .+?>', '', palavras_xml, re.U)

    # remove xml declaration necessary to load from string in etree
    palavras_xml = palavras_xml.replace('<?xml version="1.0" encoding="UTF-8"?>', '')

    # bug for output semantic roles in xml format
    palavras_xml = re.sub(r'form="(.+) <.*?"', r'form="\1"', palavras_xml, re.U)
    palavras_xml = re.sub(r'form="(.+) PU.*?"', r'form="\1"', palavras_xml, re.U)

    # bug for output semantic roles showing  £CLE
    palavras_xml = palavras_xml.replace(' £CLE', '')

    # deal with semantic roles outputed in deprel. ex:
    # head="0" deprel="PU" obj="O"
    
    palavras_xml = re.sub(r'deprel="([^§]+?)"', r'deprel="\1" srl=""', palavras_xml, re.U)

    for i in range(palavras_xml.count('§')):
        # head="0" deprel="STA §PRED" obj="O"
        palavras_xml = re.sub(r'deprel="(.+?) §(.+)?"', r'deprel="\1" srl="\2"', palavras_xml, re.U)
    try:
        parser = etree.XMLParser(remove_blank_text=True)
        tree = etree.fromstring(palavras_xml, parser)
        return tree
    except XMLSyntaxError as err:
        logger.error('XMLSyntaxError on PALAVRAS output: {} for output "{}"'.format(err, output))
    return None

In [5]:
tree = fix_palavras_malt(output)
if tree is not None:
    print(etree.tounicode(tree, pretty_print=True))

<treebank id="Arboretum">
  <body>
    <sentence>
      <word id="1" form="O" base="o" postag="pron-indef" morf="M S" extra="* " head="2" deprel="DN" srl=""/>
      <word id="2" form="menino" base="menino" postag="n" morf="M S" sem="Hbio" extra="--" head="3" deprel="S" srl="AG"/>
      <word id="3" form="jogou" base="jogar" postag="v-fin" morf="PS 3S IND VFIN" extra="fmc vH mv" head="0" deprel="STA" srl="PRED"/>
      <word id="4" form="bola" base="bola" postag="n" morf="F S" sem="cc tool food-c-h act" extra="--" head="3" deprel="Od" srl="TH"/>
      <word id="5" form="." base="--" postag="pu" morf="--" extra="--" head="0" deprel="PU" srl=""/>
    </sentence>
  </body>
</treebank>



## 2. Processing the text with PALAVRAS to generate the tokens tag

In [6]:
from lxml import etree

RELI_CORPUS = '../corpus/ReLi.xml'
CORPUS_PALAVRAS = '../corpus/ReLiPalavras.xml'

parser = etree.XMLParser(remove_blank_text=True)
reviews = etree.parse(RELI_CORPUS, parser)

PALAVRAS has its own tokenization and sentence split. It is then necessary to transfer the annotation from ReLi tokens to PALAVRAS tokens. It will be done using the functions below.

In [7]:
contractions = dict(
[('ao', ('a', 'o')),
 ('aonde', ('a', 'onde')),
 ('aos', ('a', 'os')),
 ('comigo', ('com', 'mim')),
 ('conosco', ('com', 'nós')),
 ('contigo', ('com', 'ti')),
 ('convosco', ('com', 'vós')),
 ('da', ('de', 'a')),
 ('dacolá', ('de', 'acolá')),
 ('dali', ('de', 'ali')),
 ('daquela', ('de', 'aquela')),
 ('daquelas', ('de', 'aquelas')),
 ('daquele', ('de', 'aquele')),
 ('daqueles', ('de', 'aqueles')),
 ('daqui', ('de', 'aqui')),
 ('daquilo', ('de', 'aquilo')),
 ('das', ('de', 'as')),
 ('daí', ('de', 'aí')),
 ('dela', ('de', 'ela')),
 ('delas', ('de', 'elas')),
 ('dele', ('de', 'ele')),
 ('deles', ('de', 'eles')),
 ('dentre', ('de', 'entre')),
 ('dessa', ('de', 'essa')),
 ('dessas', ('de', 'essas')),
 ('desse', ('de', 'esse')),
 ('desses', ('de', 'esses')),
 ('desta', ('de', 'esta')),
 ('destas', ('de', 'estas')),
 ('deste', ('de', 'este')),
 ('destes', ('de', 'estes')),
 ('disso', ('de', 'isso')),
 ('disto', ('de', 'isto')),
 ('do', ('de', 'o')),
 ('donde', ('de', 'onde')),
 ('dos', ('de', 'os')),
 ('doutra', ('de', 'outra')),
 ('doutras', ('de', 'outras')),
 ('doutro', ('de', 'outro')),
 ('doutros', ('de', 'outros')),
 ('na', ('em', 'a')),
 ('naquela', ('em', 'aquela')),
 ('naquelas', ('em', 'aquelas')),
 ('naquele', ('em', 'aquele')),
 ('naqueles', ('em', 'aqueles')),
 ('naquilo', ('em', 'aquilo')),
 ('nas', ('em', 'as')),
 ('nela', ('em', 'ela')),
 ('nelas', ('em', 'elas')),
 ('nele', ('em', 'ele')),
 ('neles', ('em', 'eles')),
 ('nessa', ('em', 'essa')),
 ('nessas', ('em', 'essas')),
 ('nesse', ('em', 'esse')),
 ('nesses', ('em', 'esses')),
 ('nesta', ('em', 'esta')),
 ('nestas', ('em', 'estas')),
 ('neste', ('em', 'este')),
 ('nestes', ('em', 'estes')),
 ('nisso', ('em', 'isso')),
 ('no', ('em', 'o')),
 ('nos', ('em', 'os')),
 ('noutra', ('em', 'outra')),
 ('noutras', ('em', 'outras')),
 ('noutro', ('em', 'outro')),
 ('noutros', ('em', 'outros')),
 ('num', ('em', 'um')),
 ('numa', ('em', 'uma')),
 ('numas', ('em', 'umas')),
 ('nuns', ('em', 'uns')),
 ('pela', ('por', 'a')),
 ('pelas', ('por', 'as')),
 ('pelo', ('por', 'o')),
 ('pelos', ('por', 'os')),
 ('à', ('a', 'a')),
 ('àquela', ('a', 'aquela')),
 ('àquelas', ('a', 'aquelas')),
 ('àquele', ('a', 'aquele')),
 ('àqueles', ('a', 'aqueles')),
 ('àquilo', ('a', 'aquilo')),
 ('às', ('a', 'as')),
 ('pra', ('para', 'a')),
 ('pro', ('para', 'o'))]
)

In [8]:
import unicodedata
# map tokenization between reli tokenization and PALAVRAS tokenization
def transfer_annotation(sentence, parsed_sentence, j):

    sentence = sentence.getchildren()
    # j is the align factor for sentence
    size_j = len(sentence)

    parsed_sentence = parsed_sentence.getchildren()
    i = 0  # align factor for parsed_sentence
    size_i = len(parsed_sentence)

    # annotation from ReLi is transfered to PALAVRAS tokenization
    aligned = False
    while not aligned:

        if i >= size_i or j >= size_j:
            aligned = True
            continue

        word = parsed_sentence[i].get('form').lower()
        candidate = sentence[j].get('form').lower()

        # align match!
        if word == candidate:
            # transfer annotation from ReLi to PALAVRAS
            parsed_sentence[i].set('obj', sentence[j].get('obj'))
            parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
            i += 1
            j += 1
            continue

        # PALAVRAS change symbom ' to `
        # PALAVRAS change symbom ... to .
        # PALAVRAS change symbom .. to .
        # PALAVRAS change symbom - to --
        # PALAVRAS remove dialog introduction symbol '-'
        # PALAVRAS remove symbol ' from the word:
        if (word == candidate.replace('\'', '`') or
                word == candidate.replace('...', '.') or
                word == candidate.replace('..', '.') or
                word == candidate.replace('-', '--') or
                word == candidate.replace('-', '') or
                word == candidate.replace('\'', '')):
            parsed_sentence[i].set('obj', sentence[j].get('obj'))
            parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
            i += 1
            j += 1
            continue

        # PALAVRAS changes admirá - las to admirar- las
        if word.endswith('-') and len(word) >= 4 and word[:-3] == candidate[:-1]:
            parsed_sentence[i].set('obj', sentence[j].get('obj'))
            parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
            i += 1
            j += 2
            continue

        # PALAVRAS sometimes put accents in the words
        w1 = unicodedata.normalize('NFD', word).encode('ascii', 'ignore')
        w2 = unicodedata.normalize('NFD', candidate).encode('ascii', 'ignore')
        if w1 == w2:
            parsed_sentence[i].set('obj', sentence[j].get('obj'))
            parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
            i += 1
            j += 1
            continue

        # tokens dont match due a collocation: hoje_em_dia
        if '_' in word:
            shift = word.count('_') + 1
            collocation = '_'.join([w.get('form').lower()
                                    for w in sentence[j: min(j + shift, size_j)]])
            if word == collocation:
                # transfer annotation from first ReLi token to PALAVRAS
                parsed_sentence[i].set('obj', sentence[j].get('obj'))
                parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
                i += 1
                j += shift
                continue

        # PALAVRAS concatenated two, three or four tokens
        if word.startswith(candidate):
            for shift in [2, 3, 4]:
                found = False
                collocation = ''.join([w.get('form').lower() for w in sentence[
                                      j: min(j + shift, size_j)]])
                if word == collocation:
                    # transfer annotation from first ReLi token to PALAVRAS
                    parsed_sentence[i].set('obj', sentence[j].get('obj'))
                    parsed_sentence[i].set(
                        'opinion', sentence[j].get('opinion'))
                    i += 1
                    j += shift
                    found = True
                    break
            if found:
                continue

        # contractions!
        if candidate in contractions:
            if (word == contractions[candidate][0] and
                    i < size_i and
                    parsed_sentence[i + 1].get('form').lower() == contractions[candidate][1]):
                parsed_sentence[i].set('obj', sentence[j].get('obj'))
                parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
                parsed_sentence[i + 1].set('obj', sentence[j].get('obj'))
                parsed_sentence[
                    i + 1].set('opinion', sentence[j].get('opinion'))

                i += 2
                j += 1

        # PALAVRAS split a token in two, three or four tokens
        if candidate.startswith(word):
            for shift in [2, 3, 4]:
                found = False
                collocation = ''.join([w.get('form').lower() for w in parsed_sentence[
                                      i: min(i + shift, size_i)]])
                if candidate == collocation:
                    for k in range(i, i + shift):
                        # transfer annotation from first ReLi token to PALAVRAS
                        parsed_sentence[k].set('obj', sentence[j].get('obj'))
                        parsed_sentence[k].set(
                            'opinion', sentence[j].get('opinion'))
                    i += shift
                    j += 1
                    found = True
                    break
            if found:
                continue

        # Unknown match, but next candidate matches, so match annotations and
        # continue...
        if (j < size_j - 1 and word == sentence[j + 1].get('form').lower()):
            parsed_sentence[i].set('obj', sentence[j + 1].get('obj'))
            parsed_sentence[i].set('opinion', sentence[j + 1].get('opinion'))
            i += 1
            j += 2
            continue

        # Unknown match, but next word matches, so match annotations and
        # continue...
        if (i < size_i - 1 and parsed_sentence[i + 1].get('form').lower() == candidate):
            parsed_sentence[i + 1].set('obj', sentence[j].get('obj'))
            parsed_sentence[i + 1].set('opinion', sentence[j].get('opinion'))
            i += 2
            j += 1
            continue

        # Unknown match, but next words match, so match annotations and
        # continue...
        if (i < size_i - 1 and 
            j < size_j - 1 and 
            parsed_sentence[i + 1].get('form').lower() == sentence[j + 1].get('form').lower()):
                parsed_sentence[i].set('obj', sentence[j].get('obj'))
                parsed_sentence[i].set('opinion', sentence[j].get('opinion'))
                i += 1
                j += 1
                continue

        # I dont know the problem
        logger.error('''Mismatch from PALAVRAS word "{0}"
                        with ReLi word "{1}".
                        Parsed Sentence: "{2}"
                        ReLi sentence: "{3}"'''.format(
            word,
            candidate,
            ' '.join([w.get('form') for w in parsed_sentence]),
            ' '.join([w.get('form') for w in sentence])))

        return -1
    
    # return the index in the sentence for which palavras tokens match
    return j

In [9]:
# detokenizer
import re
def detokenizer(sentence):
    # punctuation to keep close to the left word: . , .. ... : ? ! ;
    sentence = re.sub(r' (\.|,|\.\.|\.\.\.|:|\?|!|;)',
                      r'\1', sentence, flags=re.U)

    # punctuation to keep close to the both words: -se -me
    sentence = re.sub(r'(\w) (-) (a|as|o|os|se|me|te|vos|lhe|lha|lhes|lhas|na|nas|no|nos|la|las|lo|los)( |$)',
                      r'\1\2\3\4', sentence, flags=re.U)

    # punctuation to keep in context (x) "x" 'x'
    sentence = re.sub(r'\( ?(.*?) ?\)', r'(\1)', sentence, flags=re.U)
    sentence = re.sub(r'" ?(.*?) ?"', r'"\1"', sentence, flags=re.U)
    sentence = re.sub(r"' ?(.*?) ?'", r"'\1'", sentence, flags=re.U)

    # correct emoticons!
    sentence = sentence.replace(': )', ' :)')
    sentence = sentence.replace(': (', ' :(')

    return sentence

In [10]:
# create a new XML, so it will be easy to transfer new sentences and new tokens.
xmldoc = etree.Element('Reviews')

total_reviews = len(list(reviews.iter('Review')))
for count, review_node in enumerate(reviews.iter('Review')):
    
    new_review_node = etree.SubElement(xmldoc, 'Review')
    new_review_node.set('rid', review_node.get('rid'))
    new_review_node.set('book_title', review_node.get('book_title'))
    new_review_node.set('score', review_node.get('score'))
    new_sentences_node = etree.SubElement(new_review_node, 'sentences')

    print('Processing review {}/{}'.format(count+1, total_reviews))

    for sentence_node in review_node.iter('sentence'):

        sentence_string = sentence_node.xpath('./text/text()')[0]

        # PALAVRAS parser
        p = Popen(PALAVRAS_CMD, stdin=PIPE, stdout=PIPE, stderr=PIPE)
        (stdout, stderr) = p.communicate(input=sentence_string.encode('utf8'))

        # the script to convert from visl format to malt format
        p = Popen(PALAVRAS_MALT, stdin=PIPE, stdout=PIPE, stderr=PIPE)
        (stdout, stderr) = p.communicate(input=stdout)

        # script to keep only semantic information output
        p = Popen(PALAVRAS_EXTRA2SEM, stdin=PIPE, stdout=PIPE, stderr=PIPE)
        (stdout, stderr) = p.communicate(input=stdout)

        output = stdout.decode('utf8')

        tree = fix_palavras_malt(output)
        
        # skip this sentence in case of error
        if tree is None:
            continue
        
        index = 0 
        
        # remove old tokens node and create a new one    
        for sent_id, parsed_sentence in enumerate(tree.iter('sentence')):

            tokens_node = sentence_node.xpath('./tokens')[0]
            # since PALAVRAS has its own (unique!) tokenization, it is necessary to transfer the ReLi
            # annotation to PALAVRAS parsed sentence
            
            # index capture the position in the sentence to continue the annotation transfer.
            # this is useful in case of sentence split by PALAVRAS
            index = transfer_annotation(tokens_node, parsed_sentence, index)            
             
            if index != -1:
                
                new_sentence_node = etree.SubElement(new_sentences_node, 'sentence')
                new_sentence_node.set('id', sentence_node.get('id') + ':' + str(sent_id))
                new_sentence_node.set('place', sentence_node.get('place'))
                new_sentence_node.set('polarity', sentence_node.get('polarity'))
                
                text_node = etree.SubElement(new_sentence_node, 'text')
                
                # palavras can change some words, so it is necessary to rebuild the text node
                string = detokenizer(' '.join([word_node.get('form') for word_node in parsed_sentence.iter('word')]))
                text_node.text = string
                
                # rebuild the start, end offsets
                string_index = 0                
                for word_node in parsed_sentence.iter('word'):                    
                    word = word_node.get('form')                    
                    if word == None or len(word) ==0:
                        logger.error('Error get word for string {}'.format(string))
                        continue
                    if string_index < len(string) and string[string_index] == word[0]:
                        start = string_index
                        string_index += len(word) - 1
                    elif string_index < len(string)-1 and string[string_index + 1] == word[0]:
                        start = string_index + 1
                        string_index += len(word) - 1 + 1        
                    elif string_index < len(string)-2 and string[string_index + 2] == word[0]:
                        start = string_index + 2
                        string_index += len(word) - 1 + 2
                    else:
                        logger.error('Error computing start offsets for {} at {}'.format(word, string))
                        continue

                    if string_index < len(string) and string[string_index] == word[-1]:
                        end = string_index + 1
                        string_index += 1
                    else:
                        logger.error('Error computing end offsets for {} at {}'.format(word, string))
                        continue
                    
                    word_node.set('from', str(start))
                    word_node.set('to', str(end))
                    
                # add opinions tag
                opinions_node = etree.Element('Opinions')
                opinion_chunk = []
                for word_node in parsed_sentence.getchildren():                    
                    if word_node.get('obj') != 'O':
                        opinion_chunk.append((word_node.get('form'),
                                              word_node.get('from'),
                                              word_node.get('to'),
                                              word_node.get('obj')))
                    else:
                        if len(opinion_chunk) >= 1:
                            chunk = detokenizer(' '.join([w for w,b,e,o in opinion_chunk]))                
                            begin = opinion_chunk[0][1]
                            end = opinion_chunk[-1][2]
                            opinion = opinion_chunk[0][3]
                            if opinion == None:
                                logger.error('Error extracting opinion for {} at {}'.format(chunk, string))
                                break
                            if opinion.endswith('-'):
                                polarity = 'negative'
                            else:
                                polarity = 'positive'
                            opinion_chunk = []
                            opinion_node = etree.SubElement(opinions_node, 'Opinion')
                            opinion_node.set('target', chunk)
                            opinion_node.set('polarity', polarity)
                            opinion_node.set('from', str(begin))
                            opinion_node.set('to', str(end))
                if len(opinions_node.getchildren()) >= 1:
                    new_sentence_node.append(opinions_node)
                                                 
                new_tokens_node = etree.SubElement(new_sentence_node, 'tokens')
                for word_node in parsed_sentence.getchildren():
                    new_tokens_node.append(word_node)
    etree.ElementTree(xmldoc).write(CORPUS_PALAVRAS,encoding='utf8', xml_declaration=True, pretty_print=True)

ERROR:root:XMLSyntaxError on PALAVRAS output: Attribute srl redefined, line 11, column 131 for output "<?xml version="1.0" encoding="UTF-8"?>
<treebank id="Arboretum">
<body>

	<word id="1" form="E" base="e" postag="conj-c" morf="--" extra="*" head="3" deprel="CO"/>
	<word id="2" form="então" base="então" postag="adv" morf="--" extra="kc" head="3" deprel="fA §MNR"/>
	<word id="3" form="entramos" base="entrar" postag="v-fin" morf="PR 1P IND VFIN" extra="fmc vH mv" head="0" deprel="STA §PRED"/>
	<word id="4" form="em" base="em" postag="prp" morf="--" extra="--" head="3" deprel="As"/>
	<word id="5" form="outro" base="outro" postag="pron-indef" morf="M S" extra="diff KOMP" head="6" deprel="DN"/>
	<word id="6" form="aspecto" base="aspecto" postag="n" morf="M S" sem="ac-cat" extra="--" head="4" deprel="DP §DES §LOC"/>
	<word id="7" form="formidável" base="formidável" postag="adj" morf="M S" extra="np-close" head="6" deprel="DN"/>
	<word id="8" form="de" base="de" postag="prp" morf="--" extra

Processing review 1/1601
Processing review 2/1601
Processing review 3/1601
Processing review 4/1601
Processing review 5/1601
Processing review 6/1601
Processing review 7/1601
Processing review 8/1601
Processing review 9/1601

ERROR:root:XMLSyntaxError on PALAVRAS output: Opening and ending tag mismatch: rel line 38 and sentence, line 49, column 12 for output "<?xml version="1.0" encoding="UTF-8"?>
<treebank id="Arboretum">
<body>

	<word id="1" form="A" base="a" postag="prp" morf="--" extra="*" head="17" deprel="fA"/>
	<word id="2" form="o" base="o" postag="pron-indef" morf="M S" extra="--" head="4" deprel="DN"/>
	<word id="3" form="mesmo" base="mesmo" postag="pron-indef" morf="M S" extra="diff KOMP" head="4" deprel="DN"/>
	<word id="4" form="tempo" base="tempo" postag="n" morf="M S" sem="am temp per ac-cat wea" extra="--" head="1" deprel="DP §DES-TMP"/>
	<word id="5" form="que" base="que" postag="conj-s" morf="--" extra="clb" head="6" deprel="SUB"/>
	<word id="6" form="achei" base="achar" postag="v-fin" morf="PS 1S IND VFIN" extra="nosubj -head fmc vH mv" head="0" deprel="CJT"/>
	<word id="7" form="esse" base="esse" postag="pron-dem" morf="M S" extra="dem" head="8" deprel="DN"/>
	<word id="8" form="final" 


Processing review 10/1601

ERROR:root:XMLSyntaxError on PALAVRAS output: Attribute srl redefined, line 29, column 128 for output "<?xml version="1.0" encoding="UTF-8"?>
<treebank id="Arboretum">
<body>

	<word id="1" form="Digo-" base="dizer" postag="v-fin" morf="PR 1S IND VFIN" extra="fmc * hyfen vH mv" head="0" deprel="STA §PRED"/>
	<word id="2" form="lhes" base="eles" postag="pron-pers" morf="M/F 3P DAT" extra="--" head="1" deprel="Oi §BEN"/>
	<word id="3" form="que" base="que" postag="conj-s" morf="--" extra="clb clb-fs" head="5" deprel="SUB"/>
	<word id="4" form="não" base="não" postag="adv" morf="--" extra="--" head="5" deprel="fA §FOC"/>
	<word id="5" form="está" base="estar" postag="v-fin" morf="PR 3S IND VFIN" extra="mv" head="1" deprel="Od §RES"/>
	<word id="6" form="entre" base="entre" postag="prp" morf="--" extra="--" head="5" deprel="As"/>
	<word id="7" form="as" base="o" postag="pron-indef" morf="F P" extra="--" head="9" deprel="DN"/>
	<word id="8" form="piores" base="mau" postag="adj" morf="F P" e

...

ERROR:root:XMLSyntaxError on PALAVRAS output: Attribute srl redefined, line 21, column 137 for output "<?xml version="1.0" encoding="UTF-8"?>
<treebank id="Arboretum">
<body>

	<word id="1" form="Não" base="não" postag="adv" morf="--" extra="*" head="2" deprel="fA §FOC"/>
	<word id="2" form="li" base="ler" postag="v-fin" morf="PS 1S IND VFIN" extra="nosubj -head fmc vH mv" head="0" deprel="CJT"/>
	<word id="3" form="outros" base="outro" postag="pron-indef" morf="M P" extra="diff KOMP" head="4" deprel="DN"/>
	<word id="4" form="livros" base="livro" postag="n" morf="M P" sem="sem-r" extra="--" head="2" deprel="Od §TH"/>
	<word id="5" form="de" base="de" postag="prp" morf="--" extra="np-close" head="4" deprel="DN"/>
	<word id="6" form="o" base="o" postag="pron-indef" morf="M S" extra="--" head="7" deprel="DN"/>
	<word id="7" form="Jorge_Amado" base="Jorge_Amado" postag="prop" morf="M S" extra="hum *" head="5" deprel="DP"/>
	<word id="8" form=", PU @PU" base="--" postag="pu" morf="--" extr


Processing review 1591/1601
Processing review 1592/1601
Processing review 1593/1601
Processing review 1594/1601
Processing review 1595/1601
Processing review 1596/1601
Processing review 1597/1601

ERROR:root:XMLSyntaxError on PALAVRAS output: Opening and ending tag mismatch: np-close line 50 and sentence, line 57, column 12 for output "<?xml version="1.0" encoding="UTF-8"?>
<treebank id="Arboretum">
<body>

	<word id="1" form="Muito" base="muito" postag="adv" morf="--" extra="* quant" head="2" deprel="DA"/>
	<word id="2" form="bom" base="bom" postag="adj" morf="M S" extra="--" head="0" deprel="fCs §ATR"/>
	<word id="3" form="o" base="o" postag="pron-indef" morf="M S" extra="--" head="4" deprel="DN"/>
	<word id="4" form="livro" base="livro" postag="n" morf="M S" sem="sem-r" extra="--" head="0" deprel="S §TH"/>
	<word id="5" form="... PU @PU" base="--" postag="pu" morf="--" extra="--" head="0" deprel="PU"/>

	<word id="6" form="eu" base="eu" postag="pron-pers" morf="M 1S NOM" extra="--" head="7" deprel="S §AG"/>
	<word id="7" form="li" base="ler" postag="v-fin" morf="PS 1S IND VFIN" extra="nosubj -head fmc vH mv" head="0" deprel="CJT"/>
	<word id="8" form="já" base="já" postag="ad


Processing review 1598/1601
Processing review 1599/1601
Processing review 1600/1601
Processing review 1601/1601

ERROR:root:Mismatch from PALAVRAS word "."
                        with ReLi word "li".
                        Parsed Sentence: "."
                        ReLi sentence: "chaaaatinho .. li porque meu professor pediu :p"
ERROR:root:Mismatch from PALAVRAS word "li"
                        with ReLi word ":p".
                        Parsed Sentence: "li porque meu professor pediu :p ."
                        ReLi sentence: "chaaaatinho .. li porque meu professor pediu :p"
